In [ ]:
import datetime as dt
import json, os
import pandas as pd
from matplotlib import pyplot as plt
from collections import defaultdict
import numpy as np
from itertools import combinations
from aquabyte.accuracy_metrics import AccuracyMetricsGenerator
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.visualize import Visualizer, _normalize_world_keypoints
from aquabyte.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point
import random
import torch
from aquabyte.data_loader import KeypointsDataset, NormalizeCentered2D, ToTensor, BODY_PARTS
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from sklearn.model_selection import train_test_split
from copy import copy, deepcopy
import pyarrow.parquet as pq
from scipy.spatial import Delaunay
from pyobb.obb import OBB
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')

gen = s3_access_utils.get_matching_s3_keys('aquabyte-research', prefix='focus-distance/weekly/2019-12-23', suffixes=['.parquet'])
keys = []
for key in gen:
    keys.append(key)

f = s3_access_utils.download_from_s3('aquabyte-research', keys[0])
df = pd.read_parquet(f)

In [ ]:
def generate_center_coordinate(metadata, x_direction=True):
    metadata = json.loads(metadata)
    if x_direction:
        x = metadata['x_coord'] + 0.5 * metadata['width']
        return x
    y = metadata['y_coord'] + 0.5 * metadata['height']
    return y


df['centroid_x'] = df.left_crop_metadata.apply(lambda x: generate_center_coordinate(x, x_direction=True))
df['centroid_y'] = df.left_crop_metadata.apply(lambda x: generate_center_coordinate(x, x_direction=False))
df['width'] = df.left_crop_metadata.apply(lambda x: json.loads(x)['width'])
df['height'] = df.left_crop_metadata.apply(lambda x: json.loads(x)['height'])
df['aspect_ratio'] = df.width / df.height


In [ ]:
MAX_WIDTH, MAX_HEIGHT = 4096, 3000+1
SQUARE_SIZE = 500
x_values = list(np.arange(0, MAX_WIDTH, SQUARE_SIZE))
y_values = list(np.arange(0, MAX_HEIGHT, SQUARE_SIZE))
counts = np.zeros([len(x_values)-1, len(y_values)-1])
for x_idx in range(len(x_values)-1):
    for y_idx in range(len(y_values)-1):
        x_low, x_high = x_values[x_idx], x_values[x_idx+1]
        y_low, y_high = y_values[y_idx], y_values[y_idx+1]
        mask_x = (df.centroid_x > x_low) & (df.centroid_x < x_high)
        mask_y = (df.centroid_y > y_low) & (df.centroid_y < y_high)
        tile_mask = mask_x & mask_y
        if df[tile_mask].shape[0] > 0:
            accept_rate = df[tile_mask].shape[0] / df[tile_mask].shape[0]
        else:
            accept_rate = 0
        if accept_rate > 0.49:
            accept_rate = 0
        counts[x_idx, y_idx] = df[tile_mask].shape[0]


In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(counts.T, annot=True)
plt.title('Accept rate by Field Position')
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(df.distance_from_camera, bins=20)
plt.grid()
plt.show()

In [ ]:
x_idx, y_idx = 4, 4
mask_x = (df.centroid_x > x_values[x_idx]) & (df.centroid_x < x_values[x_idx+1])
mask_y = (df.centroid_y > y_values[y_idx]) & (df.centroid_y < y_values[y_idx+1])
tile_mask = mask_x & mask_y
depth_mask = (df.distance_from_camera > 0.65) & (df.distance_from_camera < 0.75)

In [ ]:
df[depth_mask & tile_mask & (df.aspect_ratio > 1.5) & (df.width < 2500)].left_crop_url

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(df[depth_mask & (df.aspect_ratio > 2)].width)
plt.grid()
plt.show()